# Rayleigh-Taylor Instability

We will represent an incompressible, viscous, diffusive, Boussinesq fluid to simulate a Rayleigh-Taylor instability. 

This exercise was designed for the course Waves and Instabilities in Geophysical Fluid Dynamics of the Master's Degree in Advanced Physics and Applied Mathematics, at University of the Balearic Islands (Spain).

Author: Daniel Argüeso
Email: d.argueso@uib.es

Feb-2023

## Import modules

In [ ]:
import numpy as np
from dedalus import public as de
from dedalus.extras import flow_tools
import matplotlib.pyplot as plt
import h5py
import time

## Import and set logging

In [ ]:
import logging
root = logging.root
for h in root.handlers:
    h.setLevel("INFO")
logger = logging.getLogger(__name__)

## Define the problem

### Set problem domain

In [ ]:
Lx, Ly = (2., 1.)
nx, ny = (1024, 512)

### Create bases and domain

In [ ]:
x_basis = de.Fourier('x', nx, interval = (0,Lx), dealias =3/2)
y_basis = de.Chebyshev('y', ny, interval=(-Ly/2, Ly/2), dealias=3/2)
domain = de.Domain([x_basis,y_basis], grid_dtype=np.float64)

### Set parameters

In [ ]:
Reynolds = 3e2
Schmidt = 1e2
g = 9.81

### Define the problem and the equations

In [ ]:
problem = de.IVP(domain, variables = ['p','u','uy','v','vy','rho','rhoy'])

In [ ]:
problem.parameters['Re'] = Reynolds
problem.parameters['Sc'] = Schmidt 
problem.parameters['g'] = g

These are the problem equations:

In [ ]:
problem.add_equation("dt(u) + dx(p) - 1/Re*(dx(dx(u)) + dy(uy)) = - u*dx(u) - v*dy(u)")
problem.add_equation("dt(v) + dy(p) - 1/Re*(dx(dx(v)) + dy(vy)) + g*rho = -u*dx(v) - v*vy")
problem.add_equation("dx(u) + vy = 0")
problem.add_equation("dt(rho) - 1/(Re*Sc)*(dx(dx(rho)) + dy(rhoy)) = -u*dx(rho) - v*rhoy")
problem.add_equation("vy - dy(v) = 0")
problem.add_equation("uy - dy(u) = 0")
problem.add_equation("rhoy - dy(rho) = 0")

### Define the boundary conditions

In [ ]:
problem.add_bc("left(u) = 0")
problem.add_bc("right(u) = 0")
problem.add_bc("left(v) = 0")
problem.add_bc("right(v) = 0", condition="(nx != 0)")
problem.add_bc("integ(p,'y') = 0", condition="(nx == 0)")
problem.add_bc("left(rho) = -0.1")
problem.add_bc("right(rho) = 0.1")

## Define the solver

### Timestepping

In [ ]:
ts = de.timesteppers.RK443

### Building the solver

In [ ]:
solver =  problem.build_solver(ts)

x = domain.grid(0)
y = domain.grid(1)
u = solver.state['u']
v = solver.state['v']
vy = solver.state['vy']
p = solver.state['p']
rho = solver.state['rho']
rhoy = solver.state['rhoy']

Set the solver parameters

In [ ]:
solver.stop_sim_time = 10.01
solver.stop_wall_time = np.inf
solver.stop_iteration = np.inf

Set initial timestep and CFL conditions

In [ ]:
initial_dt = 0.05*Lx/nx
cfl = flow_tools.CFL(solver,initial_dt,safety=0.5,threshold=0.05)
cfl.add_velocities(('u','v'))

### Initial conditions

Create array with noise for initial conditions

In [ ]:
gshape = domain.dist.grid_layout.global_shape(scales=1)
slices = domain.dist.grid_layout.slices(scales=1)
rand = np.random.RandomState(seed=42)
noise = rand.standard_normal(gshape)[slices]

Set the initial conditions for velocity and density

In [ ]:
a = 0.02
amp = -0.2
sigma = 0.2
flow = -1.0
u['g'] = 0
rho['g'] = 0.1*np.tanh(4*y/a)
v['g'] = noise / 40

## Solving

Prepare the variables that will be saved for analysis (this is optional)

In [ ]:
analysis = solver.evaluator.add_file_handler('analysis_tasks', sim_dt=0.1, max_writes=50)
analysis.add_task('rho')
analysis.add_task('u')
solver.evaluator.vars['Lx'] = Lx

### Plotting initial state

Make the plot for the initial state

In [ ]:
x = domain.grid(0,scales=domain.dealias)
y = domain.grid(1,scales=domain.dealias)
xm, ym = np.meshgrid(x,y)
fig, axis = plt.subplots(figsize=(8,5))
rho.set_scales(domain.dealias)
p = axis.pcolormesh(xm, ym, rho['g'].T, cmap='RdYlBu')
axis.set_title('Density')
axis.set_xlim([0,2.])
axis.set_ylim([-0.5,0.5])
plt.savefig(f'./Rayleigh-Taylor_000.png')

### Move into solving loop

In [ ]:
logger.info('Starting loop')
start_time = time.time()
nt=1
while solver.ok:
    dt = cfl.compute_dt()
    solver.step(dt)
    if solver.iteration % 5 == 0:
        p.set_array(rho['g'].T)
        axis.set_title('Density')
        fig.canvas.draw()
        plt.savefig(f'./Rayleigh-Taylor_{nt:03d}.png')
        nt+=1

### ending program with information

In [7]:
end_time = time.time()
logger.info('Run time: %f' %(end_time-start_time))
logger.info('Iterations: %i' %solver.iteration)

NameError: name 'time' is not defined